# Git Branch and Folder Structure

In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

In [3]:
conda install -c conda-forge selenium

Jupyter detected...
3 channel Terms of Service accepted
Channels:
 - conda-forge
 - defaults
Platform: win-64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.




==> WARNING: A newer version of conda exists. <==
    current version: 25.5.1
    latest version: 25.7.0

Please update conda by running

    $ conda update -n base -c defaults conda




In [4]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [5]:
# this library is to manipulate browser
from selenium import webdriver

# it allows you to work with differen versions of drivers
# We call ChromeDriver
# from webdriver_manager.chrome import ChromeDriverManager
import re
import time 
from selenium.webdriver.common.by import By

# Exercise 2

In [7]:
#2. Task Description
#Scrape all Data Science job offers from the Bumeran platform that match the following filters (using code not by hand!):

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [14]:
# driver = webdriver.Chrome( ChromeDriverManager().install() )
# Maximize window
driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.bumeran.com.pe/empleos-seniority-junior.html?landing-jovenes-profesionales=true")

wait = WebDriverWait(driver, 10)

# Click en "Fecha de publicación"
fecha_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Fecha de publicación')]")))
fecha_btn.click()

# Click en "Menor a 15 días" - Texto exacto
opcion = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[text()='Menor a 15 días']")))
opcion.click()

# Click en "Área" (esto abre el buscador)
area_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Área')]")))
area_btn.click()
opcion = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[text()='Tecnología, Sistemas y Telecomunicaciones']")))
opcion.click()

# Click en "Subárea"
subarea_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Subárea')]")))
subarea_btn.click()

# Click en "Programación"
programacion_option = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[text()='Programación']")))
programacion_option.click()

# Escribir "Departamento"
depto_input = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Departamento')]")))
depto_input.click()
opcion = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[text()='Lima']")))
opcion.click()

In [15]:

wait = WebDriverWait(driver, 15)

# --- SCRAPEO ---
with open("bumeran_jobs.csv", "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["Título", "Empresa", "Ubicación", "Fecha", "Link"])  # encabezados
    
    # Espera a que aparezcan las ofertas
    ofertas = wait.until(
        EC.presence_of_all_elements_located((By.XPATH, "//a[.//h2]"))
    )
    print(f"Se encontraron {len(ofertas)} ofertas")

    for oferta in ofertas:
        try:
            titulo = oferta.find_element(By.TAG_NAME, "h2").text
        except:
            titulo = ""

        try:
            empresa = oferta.find_element(By.TAG_NAME, "h3").text
        except:
            empresa = ""

        try:
            ubicacion = oferta.find_elements(By.XPATH, ".//span[contains(@class,'sc')]")
            ubicacion = ubicacion[0].text if ubicacion else ""
        except:
            ubicacion = ""

        try:
            fecha = oferta.find_elements(By.XPATH, ".//span[contains(@class,'sc')]")
            fecha = fecha[-1].text if fecha else ""
        except:
            fecha = ""

        link = oferta.get_attribute("href") or ""

        writer.writerow([titulo, empresa, ubicacion, fecha, link])

driver.quit()

Se encontraron 16 ofertas


We start by building the necessary functions to loop over each geographical subdivision and get the tables for every district. The final step of this algorithm is parse the table and add it to a collection of tables that we will later concatenate with `pandas.concat()`. We will ignore the "--SELECCIONE--" option, as it does not contain district-level information.

In [167]:
from selenium.webdriver.remote.webdriver import WebDriver

def scrape_district_table(driver: WebDriver, all_tables: list[pd.DataFrame], region_name: str, province_name: str, district_id: int):
        districts = driver.find_element(By.ID, "cod_dist")
        district = districts.find_elements(By.TAG_NAME, "option")[district_id]
        district_name = district.text

        if district_name == "-- SELECCIONE --":
                return
        
        district.click()
        table_element = driver.find_element(By.ID, "table1")
        table = parse_table_from_element(table_element)
        table["DEPARTAMENTO"] = region_name
        table["PROVINCIA"] = province_name
        table["DISTRITO"] = district_name
        all_tables.append(table)
        return




In [168]:
!pip install tqdm

In [176]:
!pip install ipywidgets

In [174]:
!pip install jupyter

Now we make a function that will loop over all districts within each province. We must keep in mind that every time we click on a new distric, we must reload the list district options, as it becomes stale. We do this in the function for the district level. In the function for the province level, we only load the list of district options to get the exact number of districts we must loop over. We also ignore the "--TODOS--" option.

In [171]:
from tqdm.notebook import tqdm

def scrape_province_tables(driver: WebDriver, all_tables: list[pd.DataFrame], region_name: str, province_id: int):
        provinces = driver.find_element(By.ID, "cod_prov")
        province = provinces.find_elements(By.TAG_NAME, "option")[province_id]
        province_name = province.text

        if province_name == "--TODOS--":
                return
        
        province.click()
        districts = driver.find_element(By.ID, "cod_dist")
        district_names = [district.text for district in districts.find_elements(By.TAG_NAME, "option")]
        num_district_options = len(district_names)

        for district_id in (pb := tqdm(range(num_district_options), leave = False)):
                pb.set_description(district_names[district_id])
                scrape_district_table(driver, all_tables, region_name, province_name, district_id)
        
        return

Finally, we define the procedure at the region level, which is very similar to the procedure for provinces.

In [179]:
def scrape_region_tables(driver: WebDriver, all_tables: list[pd.DataFrame], region_id: int):
        regions = driver.find_element(By.ID, "select_departamento")
        region = regions.find_elements(By.TAG_NAME, "option")[region_id]
        region_name = region.text

        if region_name == "--TODOS--":
                return
        
        region.click()
        provinces = driver.find_element(By.ID, "cod_prov")
        province_names = [province.text for province in provinces.find_elements(By.TAG_NAME, "option")]
        num_province_options = len(province_names)

        for province_id in (pb := tqdm(range(num_province_options), leave = False)):
                pb.set_description(province_names[province_id])
                scrape_province_tables(driver, all_tables, region_name, province_id)

In [186]:
pip install --upgrade ipywidgets jupyter

Note: you may need to restart the kernel to use updated packages.


We can now apply our functions. Scraping all regions would take a long time, so we only do it for 5 of the options

In [188]:
all_tables = []

regions = driver.find_element(By.ID, "select_departamento")
region_names = [region.text for region in regions.find_elements(By.TAG_NAME, "option")]

for region_id in (pb := tqdm(range(5), leave = False)):
        pb.set_description(region_names[region_id]) 
        scrape_region_tables(driver, all_tables, region_id)

print("FINISHED!")

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [189]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [193]:
pip install --upgrade ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [195]:
from ipywidgets import IProgress
print("IProgress importado correctamente")

ImportError: cannot import name 'IProgress' from 'ipywidgets' (c:\Users\Usuario\anaconda3\envs\selenium_env\Lib\site-packages\ipywidgets\__init__.py)

In [199]:
pip install ipywidgets jupyterlab-widgets

Note: you may need to restart the kernel to use updated packages.


In [139]:
master = pd.concat(all_tables).reset_index(drop = True)
master

ValueError: No objects to concatenate

In [118]:
master.to_csv("_data_results/presidential_election_results.csv", index = False)

The following is a version of the code that achieves the same thing but without aggregating the nested loops into functios. For the sake of having clean and understandable code, it is advised to aggregate nested loops into functions to get clearer picture of the steps our algorithms follow. A useful rule of thumb is "each function should do one thing," derived from GNU's principle "each program should do one thing and do it well." This is only a guideline and can be broken in some circumstances, but it is very helpful to keep it in mind when working with complex algorithms with nested control sequences.

In [119]:
# Store all_tables
all_tables = []

driver.get(url_1)

time.sleep(1)

resumen_general = driver.find_element( By.XPATH, '/html/body/onpe-root/onpe-home-onpe/div[1]/div/div/div/div[2]/div[2]/div/div/a/div[1]/img' )
resumen_general.click()

driver.find_elements( By.CLASS_NAME, 'btn_rojo')[1].click()

opt_peru = driver.find_element( By.XPATH, '/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div/select/option[2]')
opt_peru.click()

dept_0 = driver.find_element(By.ID, "select_departamento")
num_regions = len(dept_0.find_elements( By.TAG_NAME, "option"))

num_regions

26

In [120]:
from selenium.webdriver.support.ui import WebDriverWait

all_tables = []

for region_id in range(3):
        opt_peru = driver.find_element(By.XPATH, '/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div/select/option[2]')
        opt_peru.click()
        dept_select = driver.find_element(By.ID, "select_departamento")
        region = dept_select.find_elements( By.TAG_NAME, "option")[region_id]
        dpt_name = region.text

        if dpt_name != "--TODOS--":
                region.click()
                provinces = driver.find_element(By.ID, "cod_prov")
                num_province_options = len(provinces.find_elements(By.TAG_NAME, "option"))
                
                for province_id in range(num_province_options):
                        provinces = driver.find_element(By.ID, "cod_prov")
                        province = provinces.find_elements(By.TAG_NAME, "option")[province_id]
                        province_name = province.text
                        
                        if province_name != "--TODOS--":
                                province.click()
                                districts = driver.find_element(By.ID, "cod_dist")
                                num_district_options = len(districts.find_elements(By.TAG_NAME, "option"))

                                for district_id in range(num_district_options):
                                        districts = driver.find_element(By.ID, "cod_dist")
                                        district = districts.find_elements(By.TAG_NAME, "option")[district_id]
                                        district_name = district.text
                                        
                                        if district_name != "-- SELECCIONE --":
                                                district.click()
                                                table_element = driver.find_element(By.ID, "table1")
                                                table = parse_table_from_element(table_element)
                                                table["DEPARTAMENTO"] = dpt_name
                                                table["PROVINCIA"] = province_name
                                                table["DISTRITO"] = district_name
                                                all_tables.append(table)
                

In [121]:
pd.concat(all_tables)

,ORGANIZACIONES POLÍTICAS,TOTAL,%VÁLIDOS,%EMITIDOS,DEPARTAMENTO,PROVINCIA,DISTRITO
1,PARTIDO NACIONALISTA PERUANO,356,9.327%,6.389%,AMAZONAS,BAGUA,ARAMANGO
2,"EL FRENTE AMPLIO POR JUSTICIA, VIDA Y LIBERTAD",13,0.341%,0.233%,AMAZONAS,BAGUA,ARAMANGO
3,PARTIDO MORADO,41,1.074%,0.736%,AMAZONAS,BAGUA,ARAMANGO
4,PERU PATRIA SEGURA,4,0.105%,0.072%,AMAZONAS,BAGUA,ARAMANGO
5,VICTORIA NACIONAL,151,3.956%,2.710%,AMAZONAS,BAGUA,ARAMANGO
...,...,...,...,...,...,...,...
14,RENACIMIENTO UNIDO NACIONAL,59,0.687%,0.489%,ANCASH,YUNGAY,YUNGAY
15,PARTIDO DEMOCRATICO SOMOS PERU,114,1.328%,0.946%,ANCASH,YUNGAY,YUNGAY
16,PARTIDO POLITICO NACIONAL PERU LIBRE,3696,43.062%,30.659%,ANCASH,YUNGAY,YUNGAY
17,DEMOCRACIA DIRECTA,16,0.186%,0.133%,ANCASH,YUNGAY,YUNGAY


In [122]:
final_data = pd.concat( all_tables).reset_index( drop = True )

In [123]:
final_data.to_excel( '_data_results/presidential_election_results.xlsx' , index = False )

Now that we are all done, we quit the driver

In [124]:
driver.quit()